In [ ]:
export PERL5LIB=/gpfs/data01/glasslab/home/iholtman/software/vcftools/src/perl

###Merging of the data 

vcf-merge "/data/scratch/iholtman/human_microglia_genotyping/SNP_CHIP/Phase_5_VCF_conversion_QC_and_liftover/VCF/sorted_filtered_human_MG_samples_chr.vcf.gz" \
"/data/scratch/iholtman/human_microglia_genotyping/compare_imputation_WGS/whole_genome/merged_RNA_imputed_and_WGS_data_new_header.vcf.gz" \
| bgzip -c > "/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/merged_RNA_imputed_and_WGS_data_and_SNP_chip.vcf.gz" 


###Change the header names of the VCF file! 
## Change the names of the header in terminal
bcftools reheader merged_RNA_imputed_and_WGS_data_and_SNP_chip.vcf.gz  -s samples.txt -o merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz 
/

##Double check the header between old and new file 
bcftools query -l merged_RNA_imputed_and_WGS_data_and_SNP_chip.vcf.gz 
bcftools query -l merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz 

##Copy, unzip, sort and bgzip again 
sort -k1,1n merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf > sort_copy_merged_RNA_imputed_and_WGS_data_and_SNP_chip.vcf


vcf-sort merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf > sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf

sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz

bgzip sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf

##Index the new header file 
tabix -p vcf sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz


#####Convert to plink format, Genotype: 0,1,2 format -> 
vcftools --gzvcf sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz --plink --out plink_merged_RNA_imputed_and_WGS_data_and_SNP_chip
plink --file plink_merged_RNA_imputed_and_WGS_data_and_SNP_chip --noweb --recodeAD --out Genotype_merged_RNA_imputed_and_WGS_data_and_SNP_chip_converted_to_012_format

#Attempt to remove annoying SNPs (didn't work)
plink --file merged_RNA_imputed_and_WGS_data_and_SNP_chip --noweb --exclude annoying_snp.txt   


vcftools --gzvcf sorted_sort_copy_merged_RNA_imputed_and_WGS_data_and_SNP_chip.vcf.gz.gz --freq --chr 1 --out chr1_analysis



In [ ]:
########One time for all data at once ! 
## Needs adjustment 

import pandas
import os 
import numpy 
import seaborn 
import pickle 
import matplotlib.pyplot as plt


### Read-Data 
data = pandas.read_table(os.path.join("/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/Genotype_merged_RNA_imputed_and_WGS_data_and_SNP_chip"), sep = " ")

##Transform in numpy object, transform and convert to pandas
table = data.as_matrix()
genotype = numpy.matrix.transpose(table)

pd_genotype = pandas.DataFrame(genotype, index = data.columns, columns= ["P4", "P5", "P7", "P8", "P12", "P15", "P17", "P19","WGS_P15", "WGS_P17", "WGS_P4", "WGS_P5", "WGS_P7", "WGS_P8", "imp_P10", "imp_P11" ,"imp_P12", "imp_P14", "imp_P15", "imp_P16", "imp_P17", "imp_P18", "imp_P19", "imp_P2" ,"imp_P21" ,  "imp_P23", "imp_P24", "imp_P26", "imp_P27", "imp_P29","imp_P3" ,"imp_P31" ,"imp_P4","imp_P5" , "imp_P6", "imp_P7", "imp_P8"])
pd_genotype = pd_genotype[6:]

#print(pd_genotype[0:10])
pd_genotype.shape

    ##Select only even rows (because they are duplicate)
    pd_genotype =pd_genotype[::2]
    pd_genotype.shape

    print(pd_genotype[1:5])

    ##Safe object as pickle 
    with open(os.path.join("/data/scratch/iholtman/human_microglia_genotyping/jupyter/chr" + str(nr +1) + "_pickle"), 'wb') as f:
        pickle.dump(pd_genotype, f, pickle.HIGHEST_PROTOCOL)

    #####################################Make correlation heatmap of chr1 
    ##Remove NA values 
    pd_genotype = pandas.DataFrame.dropna(pd_genotype)
    #print(pd_genotype[1:10])
    pd_genotype.shape

    ##Calculate correlations 
    plot_data = pd_genotype.astype(float).corr()
    #print(plot_data)

    PLOT_CMAP = 'YlOrRd'
    VMIN = 0
    VMAX = 1
    fig_size=(12,12)

    ##PLot and write to files

    cm = seaborn.clustermap(plot_data, figsize=fig_size, vmin=VMIN, vmax=VMAX, cmap=PLOT_CMAP, annot=False)
    _=plt.setp(cm.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)

    plt.savefig(os.path.join("/data/scratch/iholtman/human_microglia_genotyping/jupyter/samples_genotype_correlations_imputed_WGS_chr" + str(nr+1) + ".pdf"),  bbox_inches='tight')
    plt.savefig(os.path.join("/data/scratch/iholtman/human_microglia_genotyping/jupyter/samples_genotype_correlations_imputed_WGS_chr" + str(nr+1) + ".png"),  bbox_inches='tight')




CalledProcessError: Command 'java -jar Xmx30g '/gpfs/data01/glasslab/home/iholtman/Molgenis/picard/picard.jar' GenotypeConcordance  CALL_VCF=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imputation_WGS/whole_genome/merged_RNA_imputed_and_WGS_data_new_header.vcf.gz  CALL_SAMPLE= WGS_P5 O=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imputation_WGS/TEST_concordance_call_WGS_P5_truth_WGS_P15.vcf  TRUTH_VCF=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imputation_WGS/whole_genome/merged_RNA_imputed_and_WGS_data_new_header.vcf.gz  TRUTH_SAMPLE= WGS_P15' returned non-zero exit status 1

In [5]:
########## 
import subprocess

samples = ["P4", "P5", "P7", "P8", "P12", "P15", "P17", "WGS_P15", "WGS_P17", "WGS_P4", "WGS_P5", "WGS_P7", "WGS_P8", "imp_P10", "imp_P11" ,"imp_P12", "imp_P14", "imp_P15", "imp_P16", "imp_P17", "imp_P18", "imp_P19", "imp_P2" ,"imp_P21" ,  "imp_P23", "imp_P24", "imp_P26", "imp_P27", "imp_P29","imp_P3" ,"imp_P31" ,"imp_P4","imp_P5" , "imp_P6", "imp_P7", "imp_P8"]
 
for file_x in samples:
    for file_y in samples:
        print("\n")
        terminal_code = ("java -Xmx40g -jar '/gpfs/data01/glasslab/home/iholtman/Molgenis/picard/picard.jar' GenotypeConcordance " + \
        " CALL_VCF=/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz " + \
        " CALL_SAMPLE=" + file_x + \
        " O=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/picard_concordance/concordance_call_" + file_x + "_truth_" + file_y + ".vcf " + \
        " TRUTH_VCF=/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz " + \
        " TRUTH_SAMPLE=" + file_y )
        
    print(terminal_code)
    print(subprocess.check_output(terminal_code, shell=True, stderr=subprocess.STDOUT).decode()) ##










































































java -Xmx40g -jar '/gpfs/data01/glasslab/home/iholtman/Molgenis/picard/picard.jar' GenotypeConcordance  CALL_VCF=/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz  CALL_SAMPLE=P4 O=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/picard_concordance/concordance_call_P4_truth_imp_P8.vcf  TRUTH_VCF=/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz  TRUTH_SAMPLE=imp_P8


CalledProcessError: Command 'java -Xmx40g -jar '/gpfs/data01/glasslab/home/iholtman/Molgenis/picard/picard.jar' GenotypeConcordance  CALL_VCF=/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz  CALL_SAMPLE=P4 O=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/picard_concordance/concordance_call_P4_truth_imp_P8.vcf  TRUTH_VCF=/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/sort_merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz  TRUTH_SAMPLE=imp_P8' returned non-zero exit status 1

In [ ]:
############Concordance table met Picard 
import subprocess

samples = ["WGS_P15", "WGS_P17", "WGS_P4", "WGS_P5", "WGS_P7", "WGS_P8", "imp_P10", "imp_P11" ,"imp_P12", "imp_P14", "imp_P15", "imp_P16", "imp_P17", "imp_P18", "imp_P19", "imp_P2" ,"imp_P21" ,  "imp_P23", "imp_P24", "imp_P26", "imp_P27", "imp_P29","imp_P3" ,"imp_P31" ,"imp_P4","imp_P5" , "imp_P6", "imp_P7", "imp_P8", "Chip_P5", "Chip_P7", "Chip_P8", "Chip_P12", "Chip_P15", "Chip_P17", "Chip_P19"]

#samples = ["WGS_P15", "WGS_P17", "WGS_P4", "WGS_P5", "WGS_P7", "WGS_P8", "imp_P10", "imp_P11" ,"imp_P12", "imp_P14", "imp_P15", "imp_P16", "imp_P17", "imp_P18", "imp_P19", "imp_P2" ,"imp_P21" ,  "imp_P23", "imp_P24", "imp_P26", "imp_P27", "imp_P29","imp_P3" ,"imp_P31" ,"imp_P4","imp_P5" , "imp_P6", "imp_P7", "imp_P8"]



for file_x in samples:
    for file_y in samples:
        print("\n")

        terminal_code = ("java -jar '/gpfs/data01/glasslab/home/iholtman/Molgenis/picard/picard.jar' GenotypeConcordance " + \
        " CALL_VCF='/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz' " + \
        " CALL_SAMPLE=" + file_x + \
        " O=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/picard_concordance/" + file_x + "_truth_" + file_y + ".vcf " + \
        " TRUTH_VCF='/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz' " + \
        " TRUTH_SAMPLE=" + file_y )
    
        print(terminal_code)
        print(subprocess.check_output(terminal_code, shell=True, stderr=subprocess.STDOUT).decode()) ##




In [2]:
############Concordance table met Picard 
import subprocess

samples = ["WGS_P15", "WGS_P17", "WGS_P4", "WGS_P5", "WGS_P7", "WGS_P8", "imp_P10", "imp_P11" ,"imp_P12", "imp_P14", "imp_P15", "imp_P16", "imp_P17", "imp_P18", "imp_P19", "imp_P2" ,"imp_P21" ,  "imp_P23", "imp_P24", "imp_P26", "imp_P27", "imp_P29","imp_P3" ,"imp_P31" ,"imp_P4","imp_P5" , "imp_P6", "imp_P7", "imp_P8"]
#samples = ["WGS_P15", "WGS_P17", "WGS_P4", "WGS_P5", "WGS_P7", "WGS_P8", "imp_P10", "imp_P11" ,"imp_P12", "imp_P14", "imp_P15", "imp_P16", "imp_P17", "imp_P18", "imp_P19", "imp_P2" ,"imp_P21" ,  "imp_P23", "imp_P24", "imp_P26", "imp_P27", "imp_P29","imp_P3" ,"imp_P31" ,"imp_P4","imp_P5" , "imp_P6", "imp_P7", "imp_P8", "Chip_P5", "Chip_P7", "Chip_P8", "Chip_P12", "Chip_P15", "Chip_P17", "Chip_P19"]



for file_x in samples:
    for file_y in samples:
        print("\n")

        terminal_code = ("java -jar '/gpfs/data01/glasslab/home/iholtman/Molgenis/picard/picard.jar' GenotypeConcordance " + \
        " CALL_VCF=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz " + \
        " CALL_SAMPLE= " + file_x + \
        " O=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imputation_WGS/TEST_concordance_call_" + file_x + "_truth_" + file_y + ".vcf " + \
        " TRUTH_VCF==/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz " + \
        " TRUTH_SAMPLE= " + file_y )
    
        #print(terminal_code)
        print(subprocess.check_output(terminal_code, shell=True, stderr=subprocess.STDOUT).decode()) ##



CalledProcessError: Command 'java -jar '/gpfs/data01/glasslab/home/iholtman/Molgenis/picard/picard.jar' GenotypeConcordance  CALL_VCF=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz  CALL_SAMPLE= WGS_P15 O=/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imputation_WGS/TEST_concordance_call_WGS_P15_truth_WGS_P15.vcf  TRUTH_VCF==/gpfs/data01/glasslab/data/scratch/iholtman/human_microglia_genotyping/compare_imp_WGS_snp_CHIPS/merged_RNA_imputed_and_WGS_data_and_SNP_chip_new_header.vcf.gz  TRUTH_SAMPLE= WGS_P15' returned non-zero exit status 1